In [1]:
!pip install pyarrow

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import pyarrow.parquet as pq 
import boto3

In [3]:
 parquet_file_name = r"customer_reviews_with_sentiment.parquet" 

In [4]:
 df = pd.read_parquet(parquet_file_name) 

In [5]:
 print('Rows: {0}, Columns: {1}'.format(df.shape[0],df.shape[1])) 

Rows: 96833, Columns: 16


In [6]:
 session = boto3.Session(region_name='us-east-1') 

In [7]:
 client = session.client('comprehend') 

In [8]:
# Try some examples
sentiment = client.detect_sentiment(
    Text="It's insulting that @awscloud marked an EBS volume limit increase support request as low severity but I can't do anything while I wait.",
    LanguageCode='en'
) 

In [9]:
print(sentiment)

{'Sentiment': 'NEGATIVE', 'SentimentScore': {'Positive': 0.0009363636490888894, 'Negative': 0.965487003326416, 'Neutral': 0.03050602786242962, 'Mixed': 0.0030704874079674482}, 'ResponseMetadata': {'RequestId': 'bf5f43e9-05a3-465d-a4e3-7182b9f37d70', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'bf5f43e9-05a3-465d-a4e3-7182b9f37d70', 'content-type': 'application/x-amz-json-1.1', 'content-length': '164', 'date': 'Thu, 25 Apr 2024 18:30:40 GMT'}, 'RetryAttempts': 0}}


In [10]:
%%time
# Sentiment of reviews -
# One roundtrip to comprehend service for each review
for i in range(15):
    review = df.iloc[i]['review_headline'] + ' - ' + df.iloc[i]['review_body']
    print(review)
    sentiment = client.detect_sentiment(Text=review,LanguageCode='en')
    print(sentiment['Sentiment'])
    print() 

If you need a new stove, this is a winner. - What a great stove.  What a wonderful replacement for my sort of antique.  Enjoy it every day.
POSITIVE

Five Stars - worked great
POSITIVE

Fast Shipping - Part exactly what I needed.  Saved by purchasing myself.
POSITIVE

Five Stars - Love my refrigerator! ! Keeps everything  cold..will recommend!
POSITIVE

Five Stars - No more running to the store for ice!  Works perfectly.
POSITIVE

Piece of Junk - It would not cool below 55 degrees and has now stopped working all together.  I would NOT recommend this piece of junk to anyone.
NEGATIVE

Works awesome for apt size 110 dryer - Works awesome for apt size 110 dryer. Handles load from apt size washer just fine. It does take longer to dry. Electric cost savings over a full size 220 is worth the time. Does not add much humidity unless lint filter is full.
POSITIVE

Five Stars - exactly what I wanted!
POSITIVE

Four Stars - AS advertised
NEUTRAL

but has poor insulation in the top - It works as a

In [11]:
 client_firehose = session.client('firehose') 

In [12]:
kinesis_delivery_stream_name = 'PUT-S3-zX3GC'

In [13]:
 # Push Reviews to Firehose
pattern = r'[\n\t\r]+'
for i in range(10):
    # Strip out any new line, tab and carriage return from json payload
    # Add a new line at the end to ensure firehose places each json record in a separate
    # row. without the new line, firehose simply places all records in a single line in S3.
    payload = re.sub(pattern,' ', df.iloc[i].to_json()) + "\n"

    print(payload)
    response = client_firehose.put_record(
        DeliveryStreamName=kinesis_delivery_stream_name,
        Record={
            'Data': payload
        }
    )

    print ('Response',response['ResponseMetadata']['HTTPStatusCode'])
    print()
'''if response['ResponseMetadata']['HTTPStatusCode'] != 200:
        print (response)
    else:
        print('.',end=' ')
'''

{"marketplace":"US","customer_id":"16199106","review_id":"R203HPW78Z7N4K","product_id":"B0067WNSZY","product_parent":"633038551","product_title":"FGGF3032MW Gallery Series 30\" Wide Freestanding Gas Range 5 Sealed Burners Easy Temperature Probe Express-Select Controls One-Touch Self Clean:","product_category":"Major Appliances","star_rating":5,"helpful_votes":0,"total_votes":0,"vine":"N","verified_purchase":"Y","review_headline":"If you need a new stove, this is a winner.","review_body":"What a great stove.  What a wonderful replacement for my sort of antique.  Enjoy it every day.","review_date":"2015-08-31","sentiment":"POSITIVE"}

Response 200

{"marketplace":"US","customer_id":"16374060","review_id":"R2EAIGVLEALSP3","product_id":"B002QSXK60","product_parent":"811766671","product_title":"Best Hand Clothes Wringer","product_category":"Major Appliances","star_rating":5,"helpful_votes":1,"total_votes":1,"vine":"N","verified_purchase":"Y","review_headline":"Five Stars","review_body":"wor

"if response['ResponseMetadata']['HTTPStatusCode'] != 200:\n       print (response)\n   else:\n       print('.',end=' ')\n"